In [1]:
import sys
sys.path.append("/usr/local/lib/python3.9/site-packages")

In [2]:
import nltk, pandas as pd, numpy as np
import re
from nltk.parse.corenlp import CoreNLPParser, CoreNLPDependencyParser
from nltk.tree import ParentedTree

In [3]:
dep_parser = CoreNLPDependencyParser(url='http://0.0.0.0:9000')
pos_tagger = CoreNLPParser(url='http://0.0.0.0:9000', tagtype='pos')

In [4]:
def convert_sentence (input_sent):
    # Parse sentence using Stanford CoreNLP Parser
    pos_type = pos_tagger.tag(input_sent.split())
    parse_tree, = ParentedTree.convert(list(pos_tagger.parse(input_sent.split()))[0])
    dep_type, = ParentedTree.convert(dep_parser.parse(input_sent.split()))
    return pos_type, parse_tree, dep_type

def multi_liaison (input_sent, output=['tagging','parse_tree','type_dep','spo','relation']):
    pos_type, parse_tree, dep_type = convert_sentence(input_sent)
    pos_sent = ' '.join([x[0]+'/'+x[1] for x in pos_type])
    # Extract subject, predicate and object
    subject, adjective = get_subject(parse_tree)
    predicate = get_predicate(parse_tree)
    objects = get_object(parse_tree)
    # Generate the relations between subjects and objects
    relation = get_relationship(dep_type, subject, predicate, objects)
    if 'tagging' in output:
        print('---TAGGING---')
        print(pos_sent)
        print()
    if 'parse_tree' in output:
        print('---PARSE TREE---')
        parse_tree.pretty_print()
        print()
    if 'type_dep' in output:
        print('---TYPED DEPENDENCIES---')
        li = []
        for x in dep_type.triples(): li.append(list(x))
        return li
        print()
    if 'spo' in output:
        print('---MULTI-LIAISON OUTPUT---')
        print('Subject: ',len(subject))
        for x in subject: print(' '.join(x))
        print('Predicate: ',len(predicate))
        for x in predicate: print(' '.join(x))
        print('Object: ',len(objects))
        for x in objects: print(' '.join(x))
        print()
    if 'relation' in output:
        print('---RELATIONSHIP---')
        for x in relation: print(x)

def get_subject (parse_tree):
    # Extract the nouns and adjectives from NP_subtree which is before the first / main VP_subtree
    subject, adjective = [],[]
    for s in parse_tree:
        if s.label() == 'NP':
            for t in s.subtrees(lambda y: y.label() in ['NN','NNP','NNS','NNPS','PRP']):
                # Avoid empty or repeated values
                if t.pos()[0] not in subject:
                    subject.append(t.pos()[0])
            for t in s.subtrees(lambda y: y.label().startswith('JJ')):
                if t.pos()[0] not in adjective:
                    adjective.append(t.pos()[0])
    return subject, adjective

def get_predicate (parse_tree):
    # Extract the verbs from the VP_subtree
    predicate = []
    for s in parse_tree.subtrees(lambda x: x.label() == 'VP'):
        for t in s.subtrees(lambda y: y.label().startswith('VB')):
            if t.pos()[0] not in predicate:
                predicate.append(t.pos()[0]) 
    return predicate

def get_object (parse_tree):
    # Extract the nouns from VP_NP_subtree
    objects, output = [],[]
    for s in parse_tree.subtrees(lambda x: x.label() == 'VP'):
        for t in s.subtrees(lambda y: y.label() == 'NP'):
            for u in t.subtrees(lambda z: z.label() in ['NN','NNP','NNS','NNPS','PRP$']):
                output = u.pos()[0]
                if u.left_sibling() is not None and u.left_sibling().label().startswith('JJ'):
                    output += u.left_sibling().pos()[0]
                if output not in objects:
                    objects.append(output)
    return objects

def get_relationship (dep_type, subject, predicate, objects):
    # Generate relations based on the relationship dependencies obtained from parse_tree.triples()
    subject = [x[0] for x in subject]
    predicate = [x[0] for x in predicate]
    objects = [x[0] for x in objects]     
    d1, d2, r1, r2, relation, s1, s2, subjs = [],[],[],[],[],[],[],[]
    w1, w2, output = '','',''
    for head, rel, dep in dep_type.triples():
        if rel in ['nsubj','acl:relcl','conj']:
            s1, s2 = head[0], dep[0]
            if s2 in subject and s1 in predicate:
                w1, w2 = s2, s1
            elif s2 in predicate and (s1 in subject or s1 in objects):
                w1, w2 = s1, s2
            elif s2 in subject and s1 in subject:
                subjs = [s1, s2]
            if w1 != '' and w2 != '':
                r1 = [w1, w2]
        if rel in ['dobj','prep','nmod','conj']:
            d1, d2 = head[0], dep[0]
            if d1 in objects and d2 in objects: 
                r2 = [d1,d2]
            elif d2 in objects:
                r2 = [d2]
            elif d1 in objects:
                r2 = [d1]
        if len(r1) != 0 and len(r2) != 0 and (r2[0] not in r1 and r2[-1] not in r1):
            if len(subjs) != 0:
                for n in subjs:
                    output = '-'.join([n] + r1[-1:] + r2)
                    if output not in relation:
                        relation.append(output)
            else:
                output = '-'.join(r1+r2)
                if output not in relation:
                    relation.append(output)  
    rm = [x for x in relation for y in relation if x != y and re.match(x,y)]
    for x in rm: 
        if x in relation:
            relation.remove(x)    
    return relation

In [5]:
li = multi_liaison("Sometimes I feel not tired and not sleepy, but always very angry and mad. I am sad. I am very hungry.")

---TAGGING---
Sometimes/RB I/PRP feel/VBP not/RB tired/JJ and/CC not/RB sleepy/JJ ,/, but/CC always/RB very/RB angry/JJ and/CC mad/JJ ./. I/PRP am/VBP sad/JJ ./. I/PRP am/VBP very/RB hungry/JJ ./.

---PARSE TREE---
                                                                         S                                                                    
                                              ___________________________|__________________________________________________________________   
                                             S                                        |                    |                                | 
     ________________________________________|_____                                   |                    |                                |  
    |      |                                       VP                                 |                    S                                | 
    |      |    ___________________________________|___             

In [6]:
li

[[('feel', 'VBP'), 'advmod', ('Sometimes', 'RB')],
 [('feel', 'VBP'), 'nsubj', ('I', 'PRP')],
 [('feel', 'VBP'), 'xcomp', ('tired', 'JJ')],
 [('tired', 'JJ'), 'advmod', ('not', 'RB')],
 [('tired', 'JJ'), 'conj', ('sleepy', 'JJ')],
 [('sleepy', 'JJ'), 'cc', ('and', 'CC')],
 [('sleepy', 'JJ'), 'advmod', ('not', 'RB')],
 [('feel', 'VBP'), 'punct', (',', ',')],
 [('feel', 'VBP'), 'conj', ('angry', 'JJ')],
 [('angry', 'JJ'), 'cc', ('but', 'CC')],
 [('angry', 'JJ'), 'advmod', ('always', 'RB')],
 [('angry', 'JJ'), 'advmod', ('very', 'RB')],
 [('angry', 'JJ'), 'conj', ('mad', 'JJ')],
 [('mad', 'JJ'), 'cc', ('and', 'CC')],
 [('feel', 'VBP'), 'punct', ('.', '.')],
 [('feel', 'VBP'), 'parataxis', ('sad', 'JJ')],
 [('sad', 'JJ'), 'nsubj', ('I', 'PRP')],
 [('sad', 'JJ'), 'cop', ('am', 'VBP')],
 [('feel', 'VBP'), 'punct', ('.', '.')],
 [('feel', 'VBP'), 'parataxis', ('hungry', 'JJ')],
 [('hungry', 'JJ'), 'nsubj', ('I', 'PRP')],
 [('hungry', 'JJ'), 'cop', ('am', 'VBP')],
 [('hungry', 'JJ'), 'advmod',

In [7]:
def get_behavior(li):
    behaviors = {}
    for group in li:
        if (group[0][1].startswith('JJ') or group[0][1].startswith('VB')) and group[0][0] not in behaviors:
            behaviors[group[0][0]] = []
        if group[2][1].startswith('RB') and group[0][0] in behaviors.keys():
            behaviors[group[0][0]].append(group[2][0])
    return behaviors

In [8]:
get_behavior(li)

{'feel': ['Sometimes'],
 'tired': ['not'],
 'sleepy': ['not'],
 'angry': ['always', 'very'],
 'mad': [],
 'sad': [],
 'hungry': ['very']}

In [ ]:
df = pd.read_excel("Narratives-Per-Line.xlsx")

In [ ]:
df["report"]

In [ ]:
df["behavior"] = df["report"].apply(lambda row : get_behavior(multi_liaison(row)))

In [ ]:
df

In [ ]:
df.to_csv("output.csv")

In [ ]:
df2 = pd.read_excel("News.xlsx")

In [ ]:
df2["behavior"] = df2["First Paragraph"].apply(lambda row : get_behavior(multi_liaison(str(row))))

In [ ]:
df2

In [ ]:
df2.to_csv("news_output.csv")